In [1]:
from dotenv import load_dotenv

load_dotenv()

#https://python.langchain.com/v0.2/docs/how_to/message_history/

# https://python.langchain.com/v0.2/docs/tutorials/chatbot/

True

In [2]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, MarkdownListOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory


In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke([HumanMessage(content="Hi! I'm Bob")])



AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None}, id='run-9c15f56c-7d17-428d-9824-48c45b606c6d-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

O modelo não guarda a memoria, pode ser passado diretamente na chamada: 

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 33, 'total_tokens': 45}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bf2e2fd-9ddf-4cf3-a268-8a3a58c98cd2-0', usage_metadata={'input_tokens': 33, 'output_tokens': 12, 'total_tokens': 45})

 Existe uma melhor forma de fazer isso

In [5]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
# Necessario criar um config onde especificamos o session_id
config = {"configurable": {"session_id": "abc2"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! How can I assist you today?'

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob! How can I help you today?'

In [9]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0097ff5-c169-4a21-b748-01060f4ec55b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Bob! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 33, 'total_tokens': 45}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-3adfb4a1-22fa-47e8-899c-d2201204b8b4-0', usage_metadata={'input_tokens': 33, 'output_tokens': 12, 'total_tokens': 45})])}

### Prompt templates

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [11]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}

In [12]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿Cómo puedo ayudarte hoy?'

Info: Como existe várias entradas, deve-se especificar qual será salva no histórico de menssagens 

In [13]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)


In [14]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! ¿Cómo puedo ayudarte hoy?'

## Managing Conversation History

Um conceito importante a ser entendido ao criar chatbots é como gerenciar o histórico de conversas. Se não for gerenciada, a lista de mensagens crescerá ilimitada e potencialmente estourará a janela de contexto do LLM. Portanto, é importante adicionar uma etapa que limite o tamanho das mensagens que você está passando.
O LangChain vem com alguns [auxiliares](https://python.langchain.com/v0.2/docs/how_to/#messages) integrados para gerenciar uma lista de mensagens.

In [15]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [16]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't know your name unless you tell me. What is your name?"

Encapsular com o Message History

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [18]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't know your name. What would you like me to call you?"

In [19]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked me any math problems yet. If you have one in mind, feel free to share it!"

In [20]:
import uuid

def generate_custom_uuid(username: str) -> uuid.UUID:
    # Usamos um namespace predefinido. O namespace DNS é comumente usado, mas você pode definir o seu próprio.
    namespace = uuid.NAMESPACE_DNS
    
    # Geramos o UUIDv5 baseado no namespace e no nome de usuário.
    custom_uuid = uuid.uuid5(namespace, username)
    
    return custom_uuid

# Exemplo de uso
username = "2"
custom_uuid = generate_custom_uuid(username)
print(f"UUID personalizado para {username}: {custom_uuid}")

UUID personalizado para 2: 4b166dbe-d99d-5091-abdd-95b83330ed3a


In [21]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = trimmer | model


response = chain.stream(
    [HumanMessage(content="what's my name?")],       
)
for res in response:
    print(res.content, '')


 
I'm 
 sorry 
, 
 but 
 I 
 don't 
 have 
 access 
 to 
 personal 
 information 
 about 
 users 
 unless 
 you 
 share 
 it 
 with 
 me 
. 
 How 
 can 
 I 
 assist 
 you 
 today 
? 
 
